In [1]:
import sys
sys.path.append("../client")  # Adjust if needed

from sentence_transformers import SentenceTransformer
from mariadb_vector_client import MariaDBVectorClient
from transformers import pipeline

In [2]:
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
client = MariaDBVectorClient()
retriever = client.as_langchain_retriever(embedding_model)

In [4]:
from transformers import pipeline

qa_model = pipeline(
    "text-generation",
    model="gpt2",              # Local, lightweight
    pad_token_id=50256         # Required for GPT-2
)

Device set to use cpu


In [5]:

def answer_query(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""[INST] You are a multilingual tech expert helping users choose the best AI-powered smartphone.

Use the following context to answer the question clearly and helpfully.

Context:
{context}

Question: {query}
[/INST]"""

    response = qa_model(
        prompt,
        max_new_tokens=512,
        truncation=True,
        do_sample=False,
        return_full_text=False
    )[0]["generated_text"]

    answer = response.strip()
    for phrase in ["Answer:", query.strip(), "Question:", prompt.strip()]:
        answer = answer.replace(phrase, "").strip()

    if not answer or len(answer) < 10:
        answer = "மன்னிக்கவும், இந்த கேள்விக்கு சரியான பதிலை உருவாக்க முடியவில்லை."

    return answer

In [6]:
query = "மிக சிறந்த AI கேமரா கொண்ட மொபைல் எது?"
print("✅ Answer:", answer_query(query))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Answer: மன்னிக்கவும், இந்த கேள்விக்கு சரியான பதிலை உருவாக்க முடியவில்லை.


In [7]:
pip install markdownify

Note: you may need to restart the kernel to use updated packages.


In [9]:
from markdownify import markdownify as md

html = """
<h1>📱 Best AI Camera Phones</h1>
<ul>
  <li>Samsung Galaxy S24 Ultra</li>
  <li>Google Pixel 8 Pro</li>
  <li>iPhone 15 Pro Max</li>
</ul>
"""

markdown = md(html)
print(markdown)

📱 Best AI Camera Phones

* Samsung Galaxy S24 Ultra
* Google Pixel 8 Pro
* iPhone 15 Pro Max


In [10]:
def export_markdown(query, answer):
    from markdownify import markdownify as md
    md_text = f"# ❓ Query\n{query}\n\n# ✅ Answer\n{md(answer)}"
    with open("output.md", "w", encoding="utf-8") as f:
        f.write(md_text)

In [11]:
export_markdown("மிக சிறந்த AI கேமரா கொண்ட மொபைல் எது?", "Samsung Galaxy S24 Ultra என்பது மிக சிறந்த AI கேமரா கொண்ட மொபைல் ஆகும்.")

In [12]:
with open("output.md", "r", encoding="utf-8") as f:
    print(f.read())

# ❓ Query
மிக சிறந்த AI கேமரா கொண்ட மொபைல் எது?

# ✅ Answer
Samsung Galaxy S24 Ultra என்பது மிக சிறந்த AI கேமரா கொண்ட மொபைல் ஆகும்.
